In [ ]:
!unzip combined_datasets.zip > /dev/null

In [ ]:
import os
import random
import shutil
from pathlib import Path
import zipfile

def create_finetune_subset(base_dir: str, output_zip: str, ratio: float = 0.3, seed: int = 42):
    """
    base_data/ から一定割合を抽出して finetune_subset.zip を作成する。

    Args:
        base_dir (str): 元データディレクトリのパス（例: "base_data"）
                        ┣ images/
                        ┗ labels/
        output_zip (str): 出力 zip のパス（例: "finetune_base_subset.zip"）
        ratio (float): 抽出割合（例: 0.3 → 30%）
        seed (int): ランダムシード
    """
    random.seed(seed)

    base_dir = Path(base_dir)
    images_dir = base_dir / "images"
    labels_dir = base_dir / "labels"

    subset_root = Path("finetune_subset")
    subset_images = subset_root / "images"
    subset_labels = subset_root / "labels"

    # 出力用ディレクトリを初期化
    if subset_root.exists():
        shutil.rmtree(subset_root)
    subset_images.mkdir(parents=True)
    subset_labels.mkdir(parents=True)

    # すべての画像を取得
    all_images = list(images_dir.glob("*.jpg"))
    if not all_images:
        raise ValueError(f"No .jpg images found in {images_dir}")

    random.shuffle(all_images)
    n_subset = int(len(all_images) * ratio)
    subset_imgs = all_images[:n_subset]

    # 抽出してコピー
    for img_path in subset_imgs:
        label_path = labels_dir / f"{img_path.stem}.txt"
        shutil.copy(img_path, subset_images / img_path.name)
        if label_path.exists():
            shutil.copy(label_path, subset_labels / label_path.name)

    # zip化（finetune_subset/ フォルダごと入れる）
    with zipfile.ZipFile(output_zip, "w", zipfile.ZIP_DEFLATED) as zipf:
        for folder_name, _, filenames in os.walk(subset_root):
            for filename in filenames:
                file_path = Path(folder_name) / filename
                zipf.write(file_path, file_path.relative_to(Path(".")))

    print(f"✅ {n_subset} 枚を抽出し、{output_zip} に保存しました。")

    # 一時ディレクトリ削除（zip化後に残したくない場合はコメントアウト外す）
    shutil.rmtree(subset_root)

In [ ]:

# --- 例 ---
create_finetune_subset("combined_datasets", "finetune_base_subset.zip", ratio=0.3)